# Decision Tree Classifier for a fruits dataset

In [7]:
# For Python 2/3 compatibility
from __future__ import print_function

In [2]:
# Training data consisting of 3 fruits: 'Apple', 'Grape' and 'Lemon'
training_data = [
    ['Green',3,'Mango'],
    ['Red',1,'Cherry'],
    ['Yellow',3,'Plum'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Cherry'],
    ['Blue',1,'Blueberry']
]

# Column Labels: used only to print the tree
header = ['color','diameter','label']

In [3]:
training_data

[['Green', 3, 'Mango'],
 ['Red', 1, 'Cherry'],
 ['Yellow', 3, 'Plum'],
 ['Yellow', 3, 'Mango'],
 ['Red', 1, 'Cherry'],
 ['Blue', 1, 'Blueberry']]

In [4]:
header

['color', 'diameter', 'label']

In [5]:
# Utility function to find the unique values in a column of this dataset

def unique_values(rows,col):
    vals = []
    for row in rows:
        vals.append(row[col])
    return set(vals)

# Alternative efficient method
#def unique_values(rows,col):
#    return set([row[col] for row in rows])

In [6]:
# Using utility function to determine the unique values in column 1
unique_values(training_data,0)

{'Blue', 'Green', 'Red', 'Yellow'}

In [7]:
# Using utility function to find the unique values in column 2
unique_values(training_data,1)

{1, 3}

In [8]:
# Using utility function to find the unique values in column 3
unique_values(training_data,2)

{'Blueberry', 'Cherry', 'Mango', 'Plum'}

In [9]:
#utility function for class count: count of unique values in a column

def class_counts(rows):
    """Counts the number of each type of example in a dataset"""
    counts = {}    #dictionary of label -> count
    for row in rows:
        label = row[-1]    #label is the last column; counting 'labels' column
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [10]:
# count of unique labels (fruit names)
class_counts(training_data)

{'Blueberry': 1, 'Cherry': 2, 'Mango': 2, 'Plum': 1}

In [11]:
#utility function to check if the value is a number or not

def is_numeric(value):
    """Test if a value is numeric"""
    return isinstance(value, int) or isinstance(value, float)


In [12]:
is_numeric(7)

True

In [13]:
is_numeric('red')

False

In [14]:
#Class Question is used to partition the dataset

class Question:
    def __init__(self, column,value):
        #print(column)
        #print(value)
        self.column = column
        self.value = value
        
    def match(self, example):
        #compare the feature value in the example to the feature value in the question
        val = example[self.column]
        # print(example)
        #print("val =", val)
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
       
    def __repr__(self):
        #helper method to print the question in a readable format
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return ('Is %s %s %s?' % (header[self.column], condition, str(self.value)))

In [15]:
#  Question if column 1 value is 3
Question(1,3)

Is diameter >= 3?

In [16]:
# assign question if column 0 is green to variable 'p'
p = Question(0,'Green')
p

Is color == Green?

In [17]:
#lets pick an example from the dataset and check if it matches with the question
# check if row 0 matches question in variable 'p'

example = training_data[0]
p.match(example)

True

In [18]:
#partition will partition a dataset
#for each row in the dataset, check if it matches the question.if so, add it to the True branch else to the false branch

def partition(rows, question):
    true_rows, false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
            
    return true_rows,false_rows


In [19]:
#lets partition the training data based on whether rows are red
true_rows, false_rows = partition(training_data, Question(0,'Red'))

In [20]:
true_rows

[['Red', 1, 'Cherry'], ['Red', 1, 'Cherry']]

In [21]:
false_rows

[['Green', 3, 'Mango'],
 ['Yellow', 3, 'Plum'],
 ['Yellow', 3, 'Mango'],
 ['Blue', 1, 'Blueberry']]

In [22]:
#Gini impurity calculates impurity
def gini(rows):
    counts = class_counts(rows)
    #print(counts)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        #print(len(rows))
        #print(counts[lbl])
        #print(prob_of_lbl)
        impurity = impurity - prob_of_lbl**2
        #print(prob_of_lbl**2)
    return impurity

In [23]:
#lets check gini impurity with no mixing
no_mixing = [['Mango'],['Mango']]
gini(no_mixing)

0.0

In [27]:
# now lets try gini impurity with 50% impurity i.e., 50% orange and 50% apple
some_mixing = [['Mango'],['Berry']]
gini(some_mixing)

0.5

In [28]:
# with lots of mixing
lots_of_mixing = [['Mango'],['Orange'],['Cherry'],['Grapefruit'],['Blueberry']]
gini(lots_of_mixing)

0.7999999999999998

In [30]:
# information gain = uncertainity of the starting node - weighted impurity of both the child node
def info_gain(left, right,current_uncertainity):
    p = float(len(left))/(len(left)+len(right))
    return current_uncertainity - p * gini(left) - (1-p)*gini(right)
    

In [31]:
# calculate the impurity of the training set
current_uncertainity = gini(training_data)
current_uncertainity


0.7222222222222221

In [32]:
# how much information do we get by partitioning on red 
true_rows,false_rows = partition(training_data, Question(0,'Red'))
#true_rows
#false_rows
info_gain(true_rows,false_rows,current_uncertainity)


0.30555555555555536

In [33]:
# how much information do we get by partitioning on green
true_rows,false_rows = partition(training_data,Question(0,'Green'))
info_gain(true_rows,false_rows,current_uncertainity)

0.12222222222222223

In [34]:
# we learned more using red (0.37) and less using green(.139)
# lets look at the true rows and false rows
true_rows,false_rows = partition(training_data,Question(0,'Red'))
true_rows

[['Red', 1, 'Cherry'], ['Red', 1, 'Cherry']]

In [35]:
false_rows

[['Green', 3, 'Mango'],
 ['Yellow', 3, 'Plum'],
 ['Yellow', 3, 'Mango'],
 ['Blue', 1, 'Blueberry']]

In [36]:
#lets look at true rows of green partition
true_rows,false_rows = partition(training_data, Question(0,'Green'))
true_rows

[['Green', 3, 'Mango']]

In [37]:
false_rows

[['Red', 1, 'Cherry'],
 ['Yellow', 3, 'Plum'],
 ['Yellow', 3, 'Mango'],
 ['Red', 1, 'Cherry'],
 ['Blue', 1, 'Blueberry']]

In [38]:
#find the best question to ask by iterating over every feature/value and calculating the information gain
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0])-1
    
    for col in range(n_features):   #for each feature
        values = set([row[col] for row in rows])     #unique value in the columns
        
        for val in values:          #for each value
            question = Question(col,val)
            
            #splitting the dataset
            true_rows, false_rows = partition(rows,question)
            
            #skip this split if it doesnot divide the dataset
            if len(true_rows)==0 or len(false_rows)==0:
                continue
                
            #calculate the information gain from this step
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            
            if gain >= best_gain:
                best_gain = gain
                best_question = question
                
    return best_gain, best_question
            
             

In [39]:
# determine the first question to ask for the training data
find_best_split(training_data)

(0.30555555555555536, Is color == Red?)

In [40]:
# Leaf node classifies the data
# This holds a dictionary of class (ex: 'Apple') -> number of times it appears in the rows 
#from the training data that reach this leaf

class Leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)


In [41]:
# Decision node asks a question 
# It holds reference to the question and the 2 child nodes

class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [42]:
#Build the tree

#Rules to build the tree:
#1. believe that it works
#2. Start by checking for the base case (no further information gain)
#3. Prepare for giant stack traces

def build_tree(rows):
    
    #1.Partion the dataset for each unique feature, 
    #2.find the information gain and 
    #3.return the question with highest gain
    gain, question = find_best_split(rows)
    
    #Base case: no further info gain
    #since we can ask no more questions, we will return a leaf
    if gain == 0:
        return Leaf(rows)
    
    #if we reach here, then we have found a useful feature/value to partition
    true_rows, false_rows = partition(rows, question)
    
    #Recursively build the true branch
    true_branch = build_tree(true_rows)
    
    #Recursively build the false branch
    false_branch = build_tree(false_rows)
    
    #Return a question node
    #This records the best attribute/value to ask at this point and branch to follow based on the answer
    return Decision_Node(question, true_branch, false_branch) 

In [43]:
#World's most elegant tree building function
def print_tree(node, spacing =""):
    
    #Base case: we have reached a leaf
    if isinstance(node, Leaf):
        print(spacing + 'Predict', node.predictions)
        #print (node.predictions)
        return
    
    #print the question at this node
    print(spacing + str(node.question))
    
    #call this function recursively on true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing +' ')
    
    #call this function recursively on false branch
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing +' ')
    

In [44]:
my_tree = build_tree(training_data)
print_tree(my_tree)

Is color == Red?
--> True:
 Predict {'Cherry': 2}
--> False:
 Is diameter >= 3?
 --> True:
  Is color == Yellow?
  --> True:
   Predict {'Plum': 1, 'Mango': 1}
  --> False:
   Predict {'Mango': 1}
 --> False:
  Predict {'Blueberry': 1}


In [45]:
def classify(row, node):
    #Base node: we have reached the leaf node
    if isinstance(node, Leaf):
        return node.predictions
    
    #To decide whether to follow the true branch or the false branch
    # Compare the value/feature stored in the node to the example we're considering
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row, node.false_branch)
    
    

In [46]:
#Demo
# tree predicts the first node of the dataset is an apple with confidence
classify(training_data[0],my_tree)

{'Mango': 1}

In [47]:
# A nice way to print the predictions at the leaf
def print_leaf(counts):
    total = sum(counts.values())*1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl]=str(int(counts[lbl]/total*100)) + "%"
    return probs

In [48]:
# print_leaf
print_leaf(classify(training_data[0],my_tree))

{'Mango': '100%'}

In [49]:
print_leaf(classify(training_data[1],my_tree))

{'Cherry': '100%'}

In [50]:
print_leaf(classify(training_data[2],my_tree))

{'Mango': '50%', 'Plum': '50%'}

In [53]:
print_leaf(classify(training_data[3],my_tree))

{'Mango': '50%', 'Plum': '50%'}

In [55]:
print_leaf(classify(training_data[4],my_tree))

{'Cherry': '100%'}

In [56]:
print_leaf(classify(training_data[5],my_tree))

{'Blueberry': '100%'}

In [51]:
# Test the training dataset
testing_data = [
    ['Green',3,'Mango'],
    ['Red',1,'Cherry'],
    ['Yellow',3,'Plum'],
    ['Yellow',3,'Mango'],
    ['Red',1,'Cherry'],
    ['Blue',1,'Blueberry']
]

In [52]:
for row in testing_data:
    print('Actual: %s and color: %s. Predicted: %s' % (row[-1], row[0], print_leaf(classify(row,my_tree))))

Actual: Mango and color: Green. Predicted: {'Mango': '100%'}
Actual: Cherry and color: Red. Predicted: {'Cherry': '100%'}
Actual: Plum and color: Yellow. Predicted: {'Plum': '50%', 'Mango': '50%'}
Actual: Mango and color: Yellow. Predicted: {'Plum': '50%', 'Mango': '50%'}
Actual: Cherry and color: Red. Predicted: {'Cherry': '100%'}
Actual: Blueberry and color: Blue. Predicted: {'Blueberry': '100%'}
